In [1]:
%matplotlib inline

import glob, json, argparse, os, sys
import string

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from Bio.PDB import *

sys.path.insert(0,'/data1/groups/keatinglab/swans/savinovCollaboration/inhibitory_fragments_structure_prediction')
from src.colabfold_process_output_utils import *
from src.colabfold_process_output import *
from src.analyze_predictions import *
from src.peak_prediction import *

# Import data

## Experimental peaks

In [22]:
path = '/data1/groups/keatinglab/swans/savinovCollaboration/analysis/230223_expPeakFinding_5/230223_experimentalpeaks_variablecutoff_modeabsolute_grouping25_maxgapdistance5.csv'
exp_df = pd.read_csv(path,index_col=0)
exp_df['gene'] = exp_df['gene'].str.replace('rpIL-coding-EcoliBL21DE3','rplL-coding-EcoliBL21DE3') # fix misnamed gene
print(len(exp_df))
exp_df.head(15)

81


,peak region first fragment center (aa),peak region last fragment center (aa),peak_fragment_score,gene,fragment ID,fragment start (aa),fragment end (aa),fragment center (aa),fragment length (aa),E = inhibitory effect (enrichment),sem(E),hydrophobicity_KyteDoolittle_classifier,instabIndex_Guruprasad_classifier,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,inhib_effect_cutoff
0,15.5,17.5,-5.935952,folA-coding-EcoliBL21DE3,folA-coding-EcoliBL21DE3_4_93_+,2,31,16.5,30,-5.935952,0.457483,hydrophobic,stable,negative,1.0,1.0,1.0,N-terminal,-3.0
1,32.5,32.5,-3.076845,folA-coding-EcoliBL21DE3,folA-coding-EcoliBL21DE3_52_141_+,18,47,32.5,30,-3.076845,1.075402,hydrophilic,unstable,positive,1.0,1.0,1.0,N-terminal,-3.0
2,72.5,72.5,-3.155299,folA-coding-EcoliBL21DE3,folA-coding-EcoliBL21DE3_172_261_+,58,87,72.5,30,-3.155299,1.427395,hydrophilic,unstable,negative,1.0,1.0,0.0,central,-3.0
3,88.5,90.5,-3.180808,folA-coding-EcoliBL21DE3,folA-coding-EcoliBL21DE3_226_315_+,76,105,90.5,30,-3.180808,0.439261,hydrophobic,unstable,negative,1.0,1.0,0.0,central,-3.0
4,111.5,111.5,-3.176115,folA-coding-EcoliBL21DE3,folA-coding-EcoliBL21DE3_289_378_+,97,126,111.5,30,-3.176115,1.016022,hydrophilic,stable,negative,1.0,1.0,0.0,C-terminal,-3.0
5,118.5,125.5,-5.570582,folA-coding-EcoliBL21DE3,folA-coding-EcoliBL21DE3_331_420_+,111,140,125.5,30,-5.570582,0.854611,hydrophilic,stable,negative,1.0,1.0,0.0,C-terminal,-3.0
6,139.5,142.5,-4.365602,folA-coding-EcoliBL21DE3,folA-coding-EcoliBL21DE3_382_471_+,128,157,142.5,30,-4.365602,1.292585,hydrophilic,stable,negative,1.0,1.0,0.0,C-terminal,-3.0
7,130.5,135.5,-4.585132,ftsZ-coding-EcoliBL21DE3,ftsZ-coding-EcoliBL21DE3_346_435_+,116,145,130.5,30,-4.585132,1.284617,hydrophobic,stable,positive,1.0,1.0,0.0,central,-3.0
8,164.5,187.5,-9.531838,ftsZ-coding-EcoliBL21DE3,ftsZ-coding-EcoliBL21DE3_478_567_+,160,189,174.5,30,-9.531838,1.005946,hydrophobic,stable,neutral,1.0,1.0,0.0,central,-3.0
9,188.5,202.5,-5.988661,ftsZ-coding-EcoliBL21DE3,ftsZ-coding-EcoliBL21DE3_526_615_+,176,205,190.5,30,-5.988661,1.474587,hydrophobic,stable,neutral,1.0,0.0,0.0,central,-3.0


Would like to know how many of these peaks are "new", e.g. not in the set that Andrew curated. Load the table of curated peaks and label these as "novel" if not close to those peaks

In [43]:
path = '/data1/groups/keatinglab/swans/savinovCollaboration/inhibitory_fragments_structure_prediction/data/PPI_inhibitory_fragment_peaks_AS_230114.csv'
andrew_df = pd.read_csv(path)
print(len(andrew_df))
andrew_df.head(20)

19


,protein-coding gene,protein-protein interaction inhibitory peak center (aa),30 aa fragment start (aa),30 aa fragment end (aa),protein-protein interaction partner gene(s),Experimental structure name,Protein chain in structure,Fragment chain in structure,overlap with Ncontacts peak?,Ncontacts peak center,Overlap degree (aa),Ncontacts value,weighted contacts peak center,weighted contacts value,notes
0,gyrA,26.5,12,41,gyrB,6rks_gyrA_AC_gyrB_BD.pdb,B,A,(TBD),NaN,NaN,NaN,NaN,NaN,NaN
1,gyrA,385.5,371,400,gyrA,6rks_gyrA_AC_gyrB_BD.pdb,C,A,yes,385.5,30,3.0,386.5,2.6,NaN
2,ftsZ,130.5,116,145,ftsZ,6unx_ftsZ_ABC.pdb,B,A,yes,129.5,30 (broad peak),4.0,129.5,2.3,NaN
3,ftsZ,177.5,163,192,ftsZ,6unx_ftsZ_ABC.pdb,B,A,yes,176.5,30 (broad peak),8.0,176.5,6.3,NaN
4,ftsZ,200.5,186,215,ftsZ,6unx_ftsZ_ABC.pdb,C,A,yes,201.5,30 (broad peak),15.0,201.5,11.7,NaN
5,ftsZ,272.5,258,287,ftsZ,6unx_ftsZ_ABC.pdb,C,A,yes,271.5,30 (broad peak),8.0,271.5,6.3,NaN
6,ftsZ,367.5,353,382,"ftsZ, ftsA, zipA, minC",NaN,?,A,yes (ftsZ),366.5,29,12.0,366.5,3.3,no structural data; intrinsically disordered t...
7,rplL,19.5,5,34,rplJ,3j7z_rplL_6_rplJ_5.pdb,5,6,yes (rpIL),19.5,30,10.0,19.5,2.8,NaN
8,rplL,19.5,5,34,rplL,1rqu_rplL_AB.pdb,A,C,yes (rpIL),19.5,30,10.0,19.5,2.8,both a homomeric (with rplL) and heteromeric i...
9,ssb,66.5,52,81,ssb,1eqq_ssb_ABCD.pdb,A,C,"yes (ssb, 1 copy)",71.5,25,6.0,71.5,4.9,NaN


In [24]:
andrew_df[andrew_df['protein-coding gene']=='rplL']

,protein-coding gene,protein-protein interaction inhibitory peak center (aa),30 aa fragment start (aa),30 aa fragment end (aa),protein-protein interaction partner gene(s),Experimental structure name,Protein chain in structure,Fragment chain in structure,overlap with Ncontacts peak?,Ncontacts peak center,Overlap degree (aa),Ncontacts value,weighted contacts peak center,weighted contacts value,notes
7,rplL,19.5,5,34,rplJ,3j7z_rplL_6_rplJ_5.pdb,5,6,yes (rpIL),19.5,30,10.0,19.5,2.8,NaN
8,rplL,19.5,5,34,rplL,1rqu_rplL_AB.pdb,A,C,yes (rpIL),19.5,30,10.0,19.5,2.8,both a homomeric (with rplL) and heteromeric i...


In [25]:
# Go through the experimental peaks df and check if each peak is similar to one already in the curated set
peak_type = []
for i,row in exp_df.iterrows():
    gene = row['gene'].split('-')[0]
    peak_start = row['peak region first fragment center (aa)']
    peak_end = row['peak region last fragment center (aa)']
    filt_df = andrew_df[(andrew_df['protein-coding gene']==gene)&
                        (andrew_df['protein-protein interaction inhibitory peak center (aa)']>=peak_start)&
                        (andrew_df['protein-protein interaction inhibitory peak center (aa)']<=peak_end)]
    if len(filt_df) > 0:
        print("Fragment is in curated set")
        peak_type.append('known')
    else:
        peak_type.append('novel')
exp_df['peak type'] = peak_type
exp_df.groupby('peak type').size()

Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set
Fragment is in curated set


peak type
known    14
novel    67
dtype: int64

In [26]:
andrew_df[(andrew_df['protein-coding gene']=='rplL')&
          (andrew_df['protein-protein interaction inhibitory peak center (aa)']>15.5)&
          (andrew_df['protein-protein interaction inhibitory peak center (aa)']<=31.5)]

,protein-coding gene,protein-protein interaction inhibitory peak center (aa),30 aa fragment start (aa),30 aa fragment end (aa),protein-protein interaction partner gene(s),Experimental structure name,Protein chain in structure,Fragment chain in structure,overlap with Ncontacts peak?,Ncontacts peak center,Overlap degree (aa),Ncontacts value,weighted contacts peak center,weighted contacts value,notes
7,rplL,19.5,5,34,rplJ,3j7z_rplL_6_rplJ_5.pdb,5,6,yes (rpIL),19.5,30,10.0,19.5,2.8,NaN
8,rplL,19.5,5,34,rplL,1rqu_rplL_AB.pdb,A,C,yes (rpIL),19.5,30,10.0,19.5,2.8,both a homomeric (with rplL) and heteromeric i...


## Predicted peaks (alphafold)

In [27]:
path = '/data1/groups/keatinglab/swans/savinovCollaboration/analysis/230214_peakPrediction_4/230223_alphafoldfragment_predictions.csv'
pred_df = pd.read_csv(path,index_col=0)
pred_df['gene'] = pred_df['gene'].str.replace('rpIL-coding-EcoliBL21DE3','rplL-coding-EcoliBL21DE3') # fix misnamed gene
print(len(pred_df))
pred_df.head()

202


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)
28,folA_1-159_folA_35-64,35,49.5,64,0.481738,87.872333,7,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.372169,...,NaN,NaN,NaN,NaN,NaN,"[(<Residue PHE het= resseq=140 icode= >, <Res...",1,1,49.5,49.5
85,folA_1-159_folA_92-121,92,106.5,121,0.487784,86.148333,7,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.414489,...,neutral,1.0,1.0,0.0,C-terminal,"[(<Residue PRO het= resseq=130 icode= >, <Res...",2,1,106.5,106.5
125,FtsZ_10-316_FtsZ_2-31,2,16.5,31,0.539908,82.679000,6,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.239451,...,negative,1.0,1.0,0.0,N-terminal,"[(<Residue ARG het= resseq=41 icode= >, <Resi...",7,1,16.5,16.5
166,FtsZ_10-316_FtsZ_43-72,43,57.5,72,0.700501,88.043333,10,/data1/groups/keatinglab/swans/savinovCollabor...,both,7.005011,...,positive,1.0,1.0,0.0,N-terminal,"[(<Residue ALA het= resseq=2 icode= >, <Resid...",6,16,54.5,80.5
289,FtsZ_10-316_FtsZ_166-195,166,180.5,195,0.802524,88.135333,9,/data1/groups/keatinglab/swans/savinovCollabor...,both,7.222718,...,neutral,1.0,0.0,0.0,central,"[(<Residue LEU het= resseq=263 icode= >, <Res...",5,22,164.5,186.5


In [28]:
pred_df.keys()

Index(['fragment_name', 'fragment start (aa)', 'fragment center (aa)',
       'fragment end (aa)', 'iptm', 'plddt', 'n_contacts', 'path', '_merge',
       'weighted_contacts', 'gene', 'condition', 'fragment ID',
       'fragment length (aa)', 'E = inhibitory effect (enrichment)', 'sem(E)',
       'hydrophobicity_KyteDoolittle_classifier',
       'instabIndex_Guruprasad_classifier', 'charge_Lehninger_classifier',
       'frag_contains_alphaHelix', 'frag_contains_betaStrand',
       'frag_contains_Turn', 'frag_centroid_fractional_position_classifier',
       'contact_set', 'cluster', 'cluster_size',
       'cluster first fragment center (aa)',
       'cluster last fragment center (aa)'],
      dtype='object')

# Novel peak prediction

Of the experimentally determined fragments for which there is no known structural evidence, which have a binding structure predicted by alphafold?

In [29]:
exp_df.gene.unique()

array(['folA-coding-EcoliBL21DE3', 'ftsZ-coding-EcoliBL21DE3',
       'groL-coding-EcoliBL21DE3', 'gyrA-coding-EcoliBL21DE3',
       'ileS-coding-EcoliBL21DE3', 'lptG-coding-EcoliBL21DE3',
       'rplL-coding-EcoliBL21DE3', 'rpoB-coding-EcoliBL21DE3',
       'ssb-coding-EcoliBL21DE3'], dtype=object)

In [30]:
pred_df.gene.unique()

array(['folA-coding-EcoliBL21DE3', 'ftsZ-coding-EcoliBL21DE3',
       'groL-coding-EcoliBL21DE3', 'gyrA-coding-EcoliBL21DE3',
       'lptG-coding-EcoliBL21DE3', 'rplL-coding-EcoliBL21DE3',
       'ssb-coding-EcoliBL21DE3'], dtype=object)

In [31]:
pred_df.condition.unique()

array(['30aa_monomer_folA', '30aa_monomer_fstZ', '30aa_dimer_groL',
       '30aa_monomer_groL', '30aa_monomer_groS', '30aa_monomer_gyrA',
       '30aa_monomer_gyrB', '30aa_monomer_lptF', '30aa_monomer_lptG',
       '30aa_monomer_rplL', '30aa_dimer_ssb', '30aa_monomer_ssb',
       '30aa_tetramer_ssb'], dtype=object)

In [32]:
benchmark_genes = {
    'folA-coding-EcoliBL21DE3',
    'ftsZ-coding-EcoliBL21DE3',
    'groL-coding-EcoliBL21DE3',
    'gyrA-coding-EcoliBL21DE3',
    'lptG-coding-EcoliBL21DE3',
    'rplL-coding-EcoliBL21DE3',
    'ssb-coding-EcoliBL21DE3'
}
benchmark_conditions = {
    '30aa_monomer_folA', '30aa_monomer_fstZ',
    '30aa_monomer_groL', '30aa_monomer_groS', '30aa_monomer_gyrA',
    '30aa_monomer_gyrB', '30aa_monomer_rplL', '30aa_dimer_ssb',
    '30aa_monomer_ssb'
}

An experimental inhibitory fragment is predicted by alphafold if the representative fragment (i.e. the fragment with the highest inhibitory effect in the peak) is within the range of the alphafold peak cluster

In [33]:
pred_df.columns

Index(['fragment_name', 'fragment start (aa)', 'fragment center (aa)',
       'fragment end (aa)', 'iptm', 'plddt', 'n_contacts', 'path', '_merge',
       'weighted_contacts', 'gene', 'condition', 'fragment ID',
       'fragment length (aa)', 'E = inhibitory effect (enrichment)', 'sem(E)',
       'hydrophobicity_KyteDoolittle_classifier',
       'instabIndex_Guruprasad_classifier', 'charge_Lehninger_classifier',
       'frag_contains_alphaHelix', 'frag_contains_betaStrand',
       'frag_contains_Turn', 'frag_centroid_fractional_position_classifier',
       'contact_set', 'cluster', 'cluster_size',
       'cluster first fragment center (aa)',
       'cluster last fragment center (aa)'],
      dtype='object')

In [41]:
# experimental peak is covered if the center (fragment with max inhib effect)
# is within the range of the alphafold predictions

exp_filt_df = exp_df[(exp_df['gene'].isin(benchmark_genes))&(exp_df['peak type']=='novel')].copy(deep=True)
pred_filt_df = pred_df[pred_df['condition'].isin(benchmark_conditions)]
print(f"{len(exp_filt_df)} experimental peaks and {len(pred_filt_df)} predicted")
count,covered = 0,0
covering_list = []
covered_list = []
for i,row in exp_filt_df.iterrows():
    print(row['gene'],row['fragment center (aa)'])
    frag_center_aa = row['fragment center (aa)']
    pred_covered = pred_df[(frag_center_aa>=pred_df['cluster first fragment center (aa)'])&
                        (frag_center_aa<=pred_df['cluster last fragment center (aa)'])&
                        (pred_df['gene']==row['gene'])]
    print(len(pred_covered))
    display(pred_covered.head(5))
    covering_list.append(list(pred_covered['fragment_name']))
    covered_list.append(True if len(pred_covered['fragment_name'])>0 else False)
    if len(pred_covered) > 0:
        covered += 1
    count += 1

exp_filt_df['covering'] = covering_list
exp_filt_df['covered'] = covered_list
print(f"{count} experimental fragments with significant inhibition and {covered} overlapping alphafold predictions")

30 experimental peaks and 78 predicted
folA-coding-EcoliBL21DE3 16.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


folA-coding-EcoliBL21DE3 32.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


folA-coding-EcoliBL21DE3 72.5
0


...

lptG-coding-EcoliBL21DE3 76.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


lptG-coding-EcoliBL21DE3 291.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


30 experimental fragments with significant inhibition and 3 overlapping alphafold predictions


In [42]:
exp_filt_df[exp_filt_df['covered']==True][['gene','fragment center (aa)','covering']]

,gene,fragment center (aa),covering
10,ftsZ-coding-EcoliBL21DE3,210.5,[FtsZ_10-316_FtsZ_187-216]
11,ftsZ-coding-EcoliBL21DE3,258.5,[FtsZ_10-316_FtsZ_266-295]
28,gyrA-coding-EcoliBL21DE3,137.5,[gyrB1copies_1-804_gyrA_123-152]


In [36]:
# relaxed definition
# experimental peak is covered if the center (fragment with max inhib effect)
# is within the range of the alphafold predictions + res_dist

res_dist = 15
exp_filt_df = exp_df[(exp_df['gene'].isin(benchmark_genes))&(exp_df['peak type']=='novel')].copy(deep=True)
pred_filt_df = pred_df[pred_df['condition'].isin(benchmark_conditions)]
print(f"{len(exp_filt_df)} experimental peaks and {len(pred_filt_df)} predicted")
count,covered = 0,0
covering_list = []
covered_list = []
for i,row in exp_filt_df.iterrows():
    print(row['gene'],row['fragment center (aa)'])
    frag_center_aa = row['fragment center (aa)']
    pred_covered = pred_df[(frag_center_aa+res_dist>=pred_df['cluster first fragment center (aa)'])&
                        (frag_center_aa-res_dist<=pred_df['cluster last fragment center (aa)'])&
                        (pred_df['gene']==row['gene'])]
    print(len(pred_covered))
    display(pred_covered.head(5))
    covering_list.append(list(pred_covered['fragment_name']))
    covered_list.append(True if len(pred_covered['fragment_name'])>0 else False)
    if len(pred_covered) > 0:
        covered += 1
    count += 1

exp_filt_df['covering'] = covering_list
exp_filt_df['covered'] = covered_list
print(f"{count} experimental fragments with significant inhibition and {covered} overlapping alphafold predictions")

30 experimental peaks and 78 predicted
folA-coding-EcoliBL21DE3 16.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


folA-coding-EcoliBL21DE3 32.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


folA-coding-EcoliBL21DE3 72.5
0


...

lptG-coding-EcoliBL21DE3 76.5
7


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)
3729,lptF1copies_1-366_lptG_56-69,56,62.5,69,0.656984,61.193000,7,/data1/groups/keatinglab/swans/savinovCollabor...,both,4.598891,...,neutral,1.0,0.0,0.0,N-terminal,"[(<Residue SER het= resseq=315 icode= >, <Res...",28,2,60.5,62.5
3730,lptF1copies_1-366_lptG_57-70,57,63.5,70,0.705597,61.093667,5,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.527984,...,neutral,1.0,0.0,0.0,N-terminal,"[(<Residue LEU het= resseq=311 icode= >, <Res...",29,1,63.5,63.5
3738,lptF1copies_1-366_lptG_65-78,65,71.5,78,0.589526,57.870000,6,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.537157,...,neutral,1.0,0.0,0.0,N-terminal,"[(<Residue ILE het= resseq=25 icode= >, <Resi...",30,1,71.5,71.5
3742,lptF1copies_1-366_lptG_69-82,69,75.5,82,0.645626,61.062333,5,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.228132,...,neutral,1.0,0.0,0.0,N-terminal,"[(<Residue ILE het= resseq=25 icode= >, <Resi...",31,1,75.5,75.5
3390,lptG_1-360_lptG_64-77,64,70.5,77,0.628744,79.219000,8,/data1/groups/keatinglab/swans/savinovCollabor...,both,5.029955,...,neutral,1.0,0.0,0.0,N-terminal,"[(<Residue MET het= resseq=25 icode= >, <Resi...",4,1,70.5,70.5


lptG-coding-EcoliBL21DE3 291.5
2


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)
3953,lptF1copies_1-366_lptG_280-293,280,286.5,293,0.651740,60.948667,9,/data1/groups/keatinglab/swans/savinovCollabor...,both,5.865663,...,neutral,1.0,0.0,0.0,C-terminal,"[(<Residue LEU het= resseq=311 icode= >, <Res...",36,1,286.5,286.5
3973,lptF1copies_1-366_lptG_300-313,300,306.5,313,0.454267,57.564667,8,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.634135,...,positive,1.0,0.0,0.0,C-terminal,"[(<Residue LEU het= resseq=305 icode= >, <Res...",37,1,306.5,306.5


30 experimental fragments with significant inhibition and 17 overlapping alphafold predictions


In [37]:
exp_filt_df[exp_filt_df['covered']==True][['gene','fragment center (aa)','covering']]

,gene,fragment center (aa),covering
4,folA-coding-EcoliBL21DE3,111.5,[folA_1-159_folA_92-121]
10,ftsZ-coding-EcoliBL21DE3,210.5,[FtsZ_10-316_FtsZ_187-216]
11,ftsZ-coding-EcoliBL21DE3,258.5,[FtsZ_10-316_FtsZ_266-295]
15,groL-coding-EcoliBL21DE3,26.5,"[groEL2copies_1-548_groEL_1-30, groEL2copies_1..."
17,groL-coding-EcoliBL21DE3,125.5,"[groEL2copies_1-548_groEL_102-131, groEL2copie..."
21,groL-coding-EcoliBL21DE3,273.5,[groES1copies_1-97_groEL_250-279]
25,groL-coding-EcoliBL21DE3,489.5,[groEL2copies_1-548_groEL_484-513]
27,gyrA-coding-EcoliBL21DE3,89.5,"[gyrA_1-872_gyrA_47-76, gyrA_1-872_gyrA_62-91,..."
28,gyrA-coding-EcoliBL21DE3,137.5,"[gyrB1copies_1-804_gyrA_119-148, gyrB1copies_1..."
29,gyrA-coding-EcoliBL21DE3,147.5,"[gyrB1copies_1-804_gyrA_119-148, gyrB1copies_1..."


In [38]:
def range_overlap(startA,endA,startB,endB):
    # check if there is no overlap, then negate
    return not ((endA < startB) or (endB < startA))

print(range_overlap(0,10,10,20))
print(range_overlap(0,10,11,20))
print(range_overlap(10,20,1,20))
print(range_overlap(10,20,1,2))

True
False
True
False


In [46]:
# relaxed definition 2
# experimental peak is covered if the range overlaps with
# the range of the alphafold predictions 

# res_dist = 15
exp_filt_df = exp_df[(exp_df['gene'].isin(benchmark_genes))].copy(deep=True)
# exp_filt_df = exp_df[(exp_df['gene'].isin(benchmark_genes))&(exp_df['peak type']=='novel')].copy(deep=True)
pred_filt_df = pred_df[pred_df['condition'].isin(benchmark_conditions)]
print(f"{len(exp_filt_df)} experimental peaks and {len(pred_filt_df)} predicted")
count,covered = 0,0
covering_list = []
covered_list = []
for i,row in exp_filt_df.iterrows():
    print(row['gene'],row['fragment center (aa)'])
    frag_center_aa = row['fragment center (aa)']
    frag_center_aa_first = row['peak region first fragment center (aa)']
    frag_center_aa_last = row['peak region last fragment center (aa)']
    
    # slightly confusing: condition is true if there is overlap
    # achieve this by checking if there is NOT overlap and then negating that
    pred_covered = pred_df[(pred_df['gene']==row['gene'])&
                           ~((frag_center_aa_first > pred_df['cluster last fragment center (aa)'])|
                             (frag_center_aa_last  < pred_df['cluster first fragment center (aa)']))
                          ]
    print(len(pred_covered))
    display(pred_covered.head(5))
    covering_list.append(list(pred_covered['fragment_name']))
    covered_list.append(True if len(pred_covered['fragment_name'])>0 else False)
    if len(pred_covered) > 0:
        covered += 1
    count += 1

exp_filt_df['covering'] = covering_list
exp_filt_df['covered'] = covered_list
print(f"{count} experimental fragments with significant inhibition and {covered} overlapping alphafold predictions")

44 experimental peaks and 78 predicted
folA-coding-EcoliBL21DE3 16.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


folA-coding-EcoliBL21DE3 32.5
0


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)


folA-coding-EcoliBL21DE3 72.5
0


...

rplL-coding-EcoliBL21DE3 19.5
2


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)
4021,rpIL_1-121_rpIL_1-30,1,15.5,30,0.565762,81.310333,6,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.394572,...,negative,1.0,1.0,0.0,N-terminal,"[(<Residue ILE het= resseq=9 icode= >, <Resid...",2,1,15.5,15.5
4024,rpIL_1-121_rpIL_4-33,4,18.5,33,0.650886,76.554667,13,/data1/groups/keatinglab/swans/savinovCollabor...,both,8.461522,...,NaN,NaN,NaN,NaN,NaN,"[(<Residue ILE het= resseq=9 icode= >, <Resid...",1,5,18.5,28.5


ssb-coding-EcoliBL21DE3 66.5
5


,fragment_name,fragment start (aa),fragment center (aa),fragment end (aa),iptm,plddt,n_contacts,path,_merge,weighted_contacts,...,charge_Lehninger_classifier,frag_contains_alphaHelix,frag_contains_betaStrand,frag_contains_Turn,frag_centroid_fractional_position_classifier,contact_set,cluster,cluster_size,cluster first fragment center (aa),cluster last fragment center (aa)
4515,ssb4copies_1-178_ssb_44-73,44,58.5,73,0.675513,64.019000,5,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.377566,...,neutral,1.0,1.0,0.0,N-terminal,"[(<Residue LEU het= resseq=113 icode= >, <Res...",40,1,58.5,58.5
4516,ssb4copies_1-178_ssb_45-74,45,59.5,74,0.671282,62.042333,5,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.356412,...,neutral,1.0,1.0,0.0,central,"[(<Residue MET het= resseq=49 icode= >, <Resi...",30,1,59.5,59.5
4517,ssb4copies_1-178_ssb_46-75,46,60.5,75,0.674440,63.216333,14,/data1/groups/keatinglab/swans/savinovCollabor...,both,9.442162,...,neutral,1.0,1.0,0.0,central,"[(<Residue ILE het= resseq=12 icode= >, <Resi...",49,1,60.5,60.5
4519,ssb4copies_1-178_ssb_48-77,48,62.5,77,0.674205,68.619000,6,/data1/groups/keatinglab/swans/savinovCollabor...,both,4.045230,...,neutral,1.0,1.0,0.0,central,"[(<Residue LYS het= resseq=4 icode= >, <Resid...",42,1,62.5,62.5
4521,ssb4copies_1-178_ssb_50-79,50,64.5,79,0.675793,75.528333,5,/data1/groups/keatinglab/swans/savinovCollabor...,both,3.378966,...,positive,1.0,1.0,0.0,central,"[(<Residue LEU het= resseq=113 icode= >, <Res...",1,1,64.5,64.5


44 experimental fragments with significant inhibition and 16 overlapping alphafold predictions


In [47]:
exp_filt_df[exp_filt_df['covered']==True][['gene','fragment center (aa)','covering','peak type']]

,gene,fragment center (aa),covering,peak type
8,ftsZ-coding-EcoliBL21DE3,174.5,[FtsZ_10-316_FtsZ_166-195],known
9,ftsZ-coding-EcoliBL21DE3,190.5,[FtsZ_10-316_FtsZ_187-216],known
10,ftsZ-coding-EcoliBL21DE3,210.5,[FtsZ_10-316_FtsZ_187-216],novel
11,ftsZ-coding-EcoliBL21DE3,258.5,[FtsZ_10-316_FtsZ_266-295],novel
12,ftsZ-coding-EcoliBL21DE3,272.5,[FtsZ_10-316_FtsZ_266-295],known
13,ftsZ-coding-EcoliBL21DE3,367.5,[FtsZ_10-316_FtsZ_352-381],known
14,groL-coding-EcoliBL21DE3,17.5,[groEL_1-548_groEL_1-30],known
20,groL-coding-EcoliBL21DE3,243.5,"[groES1copies_1-97_groEL_238-267, groES1copies...",known
21,groL-coding-EcoliBL21DE3,273.5,[groES1copies_1-97_groEL_250-279],novel
27,gyrA-coding-EcoliBL21DE3,89.5,"[gyrB1copies_1-804_gyrA_79-108, gyrB1copies_1-...",novel


In [51]:
exp_filt_df[(exp_filt_df['covered']==True)&(exp_filt_df['peak type']=='known')][['gene','fragment center (aa)','covering']]

,gene,fragment center (aa),covering
8,ftsZ-coding-EcoliBL21DE3,174.5,[FtsZ_10-316_FtsZ_166-195]
9,ftsZ-coding-EcoliBL21DE3,190.5,[FtsZ_10-316_FtsZ_187-216]
12,ftsZ-coding-EcoliBL21DE3,272.5,[FtsZ_10-316_FtsZ_266-295]
13,ftsZ-coding-EcoliBL21DE3,367.5,[FtsZ_10-316_FtsZ_352-381]
14,groL-coding-EcoliBL21DE3,17.5,[groEL_1-548_groEL_1-30]
20,groL-coding-EcoliBL21DE3,243.5,"[groES1copies_1-97_groEL_238-267, groES1copies..."
35,gyrA-coding-EcoliBL21DE3,382.5,"[gyrA_1-872_gyrA_379-408, gyrB1copies_1-804_gy..."
55,lptG-coding-EcoliBL21DE3,323.5,[lptF1copies_1-366_lptG_313-326]
56,rplL-coding-EcoliBL21DE3,19.5,"[rpIL_1-121_rpIL_1-30, rpIL_1-121_rpIL_4-33]"
80,ssb-coding-EcoliBL21DE3,66.5,"[ssb4copies_1-178_ssb_44-73, ssb4copies_1-178_..."
